In [9]:
# Import libs
import numpy as np
import pandas as pd
import warnings
from numpy import sqrt,arccos,arcsin,sin,cos,mod,pi,mean

#********************************************************************************************************
#Math for coordinate transformations

#Conversion of Degree <--> Radians
def deg2rad(x):
    return x*pi/180
def rad2deg(x):
    return x*180/pi

# Shperical -> Cartisan coordinate
def get_cart(lon,lat,r=1):
    #phi angle measured from z-axis
    phi = pi/2.-deg2rad(float(lat))
    #theta angle measured in XY palne
    #if longitude > 360. I should be recalculated...
    #angle wraped beteween -180 to +180
    theta = deg2rad(float(mod(lon,360.)-180.))
    #conversion
    x = r*sin(phi)*cos(theta)
    y = r*sin(phi)*sin(theta)
    z = r*cos(phi)
    return (x,y,z)

# Cartisan to Spherical coordinate
def get_sph(x,y,z):
    #conversion
    r = sqrt(x*x+y*y+z*z)
    phi = arccos(z/r)
    theta = arcsin(y/r/sin(phi))
    lon = rad2deg(theta)
    #angle unwraped
    lon = 180.+lon
    lat = rad2deg(pi/2.-phi)
    if lat > 90.:
        #sin(x) is positive in I and II quadrant
        #bring back latitude value to valid range
        lat = lat-180.
    return (r,lon,lat)

# Calculate angular seperation between two points
def get_ang_sep(P1,P2):
    x1,y1,z1 = P1[0],P1[1],P1[2]
    x2,y2,z2 = P2[0],P2[1],P2[2]
    angle = (x1*x2+y1*y2+z1*z2)/sqrt(x1**2+y1**2+z1**2)/sqrt(x2**2+y2**2+z2**2)
    angle = arccos(angle)
    return rad2deg(angle)

In [43]:
# Defining class for OOP


class coords:
    
    # User i/p is latitude (lat), longitude (lon) [, distance (dist),suppress_warning]
    # Default distance 
    def __init__(self,lat=[0.],lon=[0],dist=[1.],suppress_warning=1):
        
        #Check lat lon array size
        if len(lat)!=len(lon):
            raise Exception(f"lat/lon should be of same dimension")
            
        #Check dist array size
        if len(lat)==len(lon) and len(lat)!=len(dist):
            dist = [1. for i in range(len(lat))]
            #print warning
            if not suppress_warning:
                warnings.warn("dist not set/ wrong dimension; default dist=1.")
                
        #Check latitude values
        for i in range(len(lat)):
            if lat[i] > 90. or lat[i] < -90.:
                raise Exception(f"Range error in lat[{i}]={lat[i]}; valid range -90<=lat<+90. ")
                
        #Assign class values
        self.lat = lat
        self.lon = lon
        self.dist = dist
        
        #Create list for cartisan coordinates within the class
        self.xp = [0. for i in range(len(lat))]
        self.yp = [0. for i in range(len(lat))]
        self.zp = [0. for i in range(len(lat))]
        
        
        #Calculate cartisan coordinates for given lat,lon,dist values
        for i in range(len(lat)):
                x,y,z = get_cart(lon[i],lat[i],dist[i])
                self.xp[i] = x
                self.yp[i] = y
                self.zp[i] = z
    
    #Length of coords (npts)
    def __len__(self):
        return len(self.lat)
    
    #Change coords class to pandas data frame
    def coords2pandas(self):
        tab = {"lon":self.lon,"lat":self.lat,"dist":self.dist,
               "xp":self.xp,"yp":self.yp,"zp":self.zp}
        tab = pd.DataFrame.from_dict(tab)
        return tab
    
    #prints coords as pandas DataFrame
    def show(self):
        tab = self.coords2pandas()
        return tab.round(2)
        
    #Calculate centroid
    def centroid(self):
        if len(self)<2:
            raise Exception("Need atleast 2 points to find centroid!!!")
        xc,yc,zc = mean(self.xp),mean(self.yp),mean(self.zp)
        sph_coord = get_sph(xc,yc,zc)
        return coords(lat=[sph_coord[2]],lon=[sph_coord[1]],dist=[sph_coord[0]])
    
    #Calculate angle of seperation between coords and loc
    #Note loc should be of type coords
    def ang_sep(self,loc):
        
        if type(loc)!=type(self):
            raise TypeError("loc should be 'coord'")
        
        #Check length of loc
        if len(loc)>1:
            raise Exception("loc should be 'coord' of lenght==1 ")
        
        #Some wiered error for 1 element coords
        #this fixes the list in tuple problem
        xp = np.asarray(loc.xp, dtype=np.float32)[0]
        yp = np.asarray(loc.yp, dtype=np.float32)[0]
        zp = np.asarray(loc.zp, dtype=np.float32)[0]
        P1 = (xp,yp,zp)
        
        #Calculate the anglular seperation
        sep = []
        for i in range(len(self)):
            xp1 = self.xp[i]
            yp1 = self.yp[i]
            zp1 = self.zp[i]
            P2 = (xp1,yp1,zp1)
            ang = get_ang_sep(P1,P2)
            sep.append(ang)

        tab = {"lon":[self.lon[i] for i in range(len(sep))],
              "lat":[self.lat[i] for i in range(len(sep))],
              "dist":[self.dist[i] for i in range(len(sep))],
              "ang_sep":sep}
        tab = pd.DataFrame.from_dict(tab)
        return tab

In [44]:
lon = [248,235.2,240,257.1,237.4,266.2,261.3]
lat = [-49.2,-49.1,-48.6,-48.4,-48.2,-44.4,-40.2]
a = coords(lat=lat,lon=lon)
x = a.centroid()
a.show()

,lon,lat,dist,xp,yp,zp
0,248.0,-49.2,1.0,0.24,0.61,-0.76
1,235.2,-49.1,1.0,0.37,0.54,-0.76
2,240.0,-48.6,1.0,0.33,0.57,-0.75
3,257.1,-48.4,1.0,0.15,0.65,-0.75
4,237.4,-48.2,1.0,0.36,0.56,-0.75
5,266.2,-44.4,1.0,0.05,0.71,-0.70
6,261.3,-40.2,1.0,0.12,0.76,-0.65


In [45]:
x.show()

,lon,lat,dist,xp,yp,zp
0,249.76,-47.46,0.99,0.23,0.63,-0.73


In [46]:
a.ang_sep(x)

,lon,lat,dist,ang_sep
0,248.0,-49.2,1.0,2.101835
1,235.2,-49.1,1.0,9.815516
2,240.0,-48.6,1.0,6.624915
3,257.1,-48.4,1.0,5.002948
4,237.4,-48.2,1.0,8.325281
5,266.2,-44.4,1.0,11.808115
6,261.3,-40.2,1.0,11.020033
